In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldaseqmodel

import  matplotlib.pyplot  as plt
import gensim
import numpy as np
import spacy

from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline
from rcare.readutils import DataFrameUtils as Utils

2018-08-03 15:03:50,994 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
DATA_PATHS = []
DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
DATA_PATHS.append(DATA_PATH)
#df_2011_q3 = Utils.read_data(path = DATA_PATH)

DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q4/text/'
#df_2011_q4 = Utils.read_data(path = DATA_PATH)
DATA_PATHS.append(DATA_PATH)

DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2012_Q4/text/'
#DATA_PATHSdf_2012_q4 = Utils.read_data(path = DATA_PATH)
DATA_PATHS.append(DATA_PATH)

DATA_PATHS

['data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/',
 'data/lexpredict-contraxsuite-samples/cca_2011_Q4/text/',
 'data/lexpredict-contraxsuite-samples/cca_2012_Q4/text/']

In [4]:
from nltk.corpus import stopwords
stopwordlist = stopwords.words('english')
stopwordlist = stopwordlist + ['â\x80\x9c' , '©', 'â', 'â\x80\x9d']
'''
for idx, stopword in enumerate(stopwordlist):
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True
'''    

'\nfor idx, stopword in enumerate(stopwordlist):\n    lexeme = nlp.vocab[stopword]\n    lexeme.is_stop = True\n'

In [38]:
docs, lines, tokens = [], [], []
for path in DATA_PATHS:
    df = Utils.read_data(path = path, numfiles = 2)
    for index, row in df.iterrows():    
        doc = nlp(Utils.clean_document(row['data'], rm_stop_words = True))        
        lines = []
        tokens = []           
        for w in doc:            
            # if it's not a stop word or punctuation mark, add it to our article!
            if w.text.strip() not in ('\n', '$','\n\n' , '\n\n\n', '\n\n\n\n',''  ) \
            and not w.text.startswith('â') \
            and w.lemma_ != '-PRON-'\
            and not w.is_stop and not w.is_punct and not w.like_num:
                # we add the lematized version of the word
                tokens.append(w.lemma_)               
        # if it's a new line, it means we're onto our next document
        lines.append(tokens)
            
    docs.append(lines)

2018-08-03 15:31:07,893 : INFO : Number of files 1362 in data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/
2018-08-03 15:31:07,901 : INFO : Reading file creditcardagreement_1530.PDF.txt
2018-08-03 15:31:07,904 : INFO : Appending to dataframe creditcardagreement_1530.PDF.txt
2018-08-03 15:31:07,911 : INFO : Reading file creditcardagreement_4904.pdf.txt
2018-08-03 15:31:07,913 : INFO : Appending to dataframe creditcardagreement_4904.pdf.txt
2018-08-03 15:31:07,918 : INFO : Reading file data done..
2018-08-03 15:31:10,245 : INFO : Number of files 1970 in data/lexpredict-contraxsuite-samples/cca_2011_Q4/text/
2018-08-03 15:31:10,256 : INFO : Reading file creditcardagreement_5332.pdf.txt
2018-08-03 15:31:10,259 : INFO : Appending to dataframe creditcardagreement_5332.pdf.txt
2018-08-03 15:31:10,267 : INFO : Reading file creditcardagreement_5632.pdf.txt
2018-08-03 15:31:10,272 : INFO : Appending to dataframe creditcardagreement_5632.pdf.txt
2018-08-03 15:31:10,285 : INFO : Reading file 

In [20]:
docs, lines, tokens = [], [], []
for path in DATA_PATHS:
    df = Utils.read_data(path = path, numfiles = 2)
    for index, row in df.iterrows():    
        doc = nlp(Utils.clean_document(row['data'], rm_stop_words = True))        
        lines = []
        for sent in doc.sents:
            tokens = []
            for w in sent:
                # if it's not a stop word or punctuation mark, add it to our article!
                if w.text.strip() not in ('\n', '$','\n\n' , '\n\n\n', '\n\n\n\n',''  ) \
                and not w.text.startswith('â') \
                and w.lemma_ != '-PRON-'\
                and not w.is_stop and not w.is_punct and not w.like_num:
                    # we add the lematized version of the word
                    tokens.append(w.lemma_)               
            # if it's a new line, it means we're onto our next document
            lines.append(tokens)
            
    docs.append(lines)

2018-08-03 15:12:18,181 : INFO : Number of files 1362 in data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/
2018-08-03 15:12:18,189 : INFO : Reading file creditcardagreement_1530.PDF.txt
2018-08-03 15:12:18,192 : INFO : Appending to dataframe creditcardagreement_1530.PDF.txt
2018-08-03 15:12:18,199 : INFO : Reading file creditcardagreement_4904.pdf.txt
2018-08-03 15:12:18,201 : INFO : Appending to dataframe creditcardagreement_4904.pdf.txt
2018-08-03 15:12:18,207 : INFO : Reading file data done..
2018-08-03 15:12:19,413 : INFO : Stop word part
2018-08-03 15:12:19,415 : INFO : Stop word whose
2018-08-03 15:12:19,417 : INFO : Stop word name
2018-08-03 15:12:19,418 : INFO : Stop word anyone
2018-08-03 15:12:19,420 : INFO : Stop word you
2018-08-03 15:12:19,422 : INFO : Stop word amount
2018-08-03 15:12:19,423 : INFO : Stop word otherwise
2018-08-03 15:12:19,425 : INFO : Stop word whereby
2018-08-03 15:12:19,428 : INFO : Stop word either
2018-08-03 15:12:19,429 : INFO : Stop word where

2018-08-03 15:12:19,597 : INFO : Stop word may
2018-08-03 15:12:19,598 : INFO : Stop word regarding
2018-08-03 15:12:19,600 : INFO : Stop word also
2018-08-03 15:12:19,601 : INFO : Stop word made
2018-08-03 15:12:19,603 : INFO : Stop word unless
2018-08-03 15:12:19,604 : INFO : Stop word must
2018-08-03 15:12:19,605 : INFO : Stop word upon
2018-08-03 15:12:19,606 : INFO : Stop word upon
2018-08-03 15:12:19,607 : INFO : Stop word whether
2018-08-03 15:12:19,608 : INFO : Stop word also
2018-08-03 15:12:19,610 : INFO : Stop word may
2018-08-03 15:12:19,612 : INFO : Stop word must
2018-08-03 15:12:19,613 : INFO : Stop word us
2018-08-03 15:12:19,614 : INFO : Stop word must
2018-08-03 15:12:19,615 : INFO : Stop word name
2018-08-03 15:12:19,616 : INFO : Stop word along
2018-08-03 15:12:19,617 : INFO : Stop word can
2018-08-03 15:12:19,618 : INFO : Stop word not
2018-08-03 15:12:19,620 : INFO : Stop word may
2018-08-03 15:12:19,623 : INFO : Stop word regarding
2018-08-03 15:12:19,624 : INFO 

2018-08-03 15:12:21,405 : INFO : Stop word made
2018-08-03 15:12:21,406 : INFO : Stop word one
2018-08-03 15:12:21,408 : INFO : Stop word you
2018-08-03 15:12:21,409 : INFO : Stop word another
2018-08-03 15:12:21,411 : INFO : Stop word may
2018-08-03 15:12:21,413 : INFO : Stop word name
2018-08-03 15:12:21,414 : INFO : Stop word someone
2018-08-03 15:12:21,415 : INFO : Stop word least
2018-08-03 15:12:21,416 : INFO : Stop word fifteen
2018-08-03 15:12:21,418 : INFO : Stop word made
2018-08-03 15:12:21,420 : INFO : Stop word may
2018-08-03 15:12:21,421 : INFO : Stop word us
2018-08-03 15:12:21,422 : INFO : Stop word regarding
2018-08-03 15:12:21,424 : INFO : Stop word name
2018-08-03 15:12:21,425 : INFO : Stop word well
2018-08-03 15:12:21,427 : INFO : Stop word name
2018-08-03 15:12:21,428 : INFO : Stop word make
2018-08-03 15:12:21,430 : INFO : Stop word via
2018-08-03 15:12:21,431 : INFO : Stop word name
2018-08-03 15:12:21,433 : INFO : Stop word one
2018-08-03 15:12:21,435 : INFO : 

2018-08-03 15:12:21,671 : INFO : Stop word amount
2018-08-03 15:12:21,672 : INFO : Stop word done
2018-08-03 15:12:21,675 : INFO : Stop word may
2018-08-03 15:12:21,676 : INFO : Stop word used
2018-08-03 15:12:21,678 : INFO : Stop word may
2018-08-03 15:12:21,679 : INFO : Stop word would
2018-08-03 15:12:21,680 : INFO : Stop word used
2018-08-03 15:12:21,682 : INFO : Stop word amount
2018-08-03 15:12:21,684 : INFO : Stop word may
2018-08-03 15:12:21,685 : INFO : Stop word above
2018-08-03 15:12:21,686 : INFO : Stop word may
2018-08-03 15:12:21,688 : INFO : Stop word may
2018-08-03 15:12:21,689 : INFO : Stop word someone
2018-08-03 15:12:21,690 : INFO : Stop word using
2018-08-03 15:12:21,691 : INFO : Stop word without
2018-08-03 15:12:21,694 : INFO : Stop word call
2018-08-03 15:12:21,695 : INFO : Stop word also
2018-08-03 15:12:21,697 : INFO : Stop word us
2018-08-03 15:12:21,698 : INFO : Stop word us
2018-08-03 15:12:21,700 : INFO : Stop word someone
2018-08-03 15:12:21,701 : INFO : 

2018-08-03 15:12:21,954 : INFO : Stop word us
2018-08-03 15:12:21,956 : INFO : Stop word so
2018-08-03 15:12:21,958 : INFO : Stop word except
2018-08-03 15:12:21,960 : INFO : Stop word whether
2018-08-03 15:12:21,962 : INFO : Stop word another
2018-08-03 15:12:21,963 : INFO : Stop word must
2018-08-03 15:12:21,965 : INFO : Stop word anything
2018-08-03 15:12:21,966 : INFO : Stop word else
2018-08-03 15:12:21,968 : INFO : Stop word may
2018-08-03 15:12:21,970 : INFO : Stop word may
2018-08-03 15:12:21,973 : INFO : Stop word may
2018-08-03 15:12:21,974 : INFO : Stop word two
2018-08-03 15:12:21,976 : INFO : Stop word may
2018-08-03 15:12:21,977 : INFO : Stop word otherwise
2018-08-03 15:12:21,979 : INFO : Stop word together
2018-08-03 15:12:21,981 : INFO : Stop word unless
2018-08-03 15:12:21,983 : INFO : Stop word whether
2018-08-03 15:12:21,985 : INFO : Stop word may
2018-08-03 15:12:21,989 : INFO : Stop word may
2018-08-03 15:12:21,991 : INFO : Stop word may
2018-08-03 15:12:21,993 : 

...

2018-08-03 15:12:26,032 : INFO : Stop word to
2018-08-03 15:12:26,033 : INFO : Stop word must
2018-08-03 15:12:26,034 : INFO : Stop word anyone
2018-08-03 15:12:26,038 : INFO : Stop word us
2018-08-03 15:12:26,039 : INFO : Stop word is
2018-08-03 15:12:26,040 : INFO : Stop word first
2018-08-03 15:12:26,041 : INFO : Stop word amount
2018-08-03 15:12:26,042 : INFO : Stop word even
2018-08-03 15:12:26,044 : INFO : Stop word may
2018-08-03 15:12:26,045 : INFO : Stop word amount
2018-08-03 15:12:26,046 : INFO : Stop word due
2018-08-03 15:12:26,048 : INFO : Stop word two
2018-08-03 15:12:26,050 : INFO : Stop word must
2018-08-03 15:12:26,051 : INFO : Stop word made
2018-08-03 15:12:26,053 : INFO : Stop word within
2018-08-03 15:12:26,054 : INFO : Stop word within
2018-08-03 15:12:26,056 : INFO : Stop word must
2018-08-03 15:12:26,086 : INFO : Number of files 1506 in data/lexpredict-contraxsuite-samples/cca_2012_Q4/text/
2018-08-03 15:12:26,096 : INFO : Reading file creditcardagreement_6281

2018-08-03 15:12:27,120 : INFO : Stop word amount
2018-08-03 15:12:27,121 : INFO : Stop word amount
2018-08-03 15:12:27,122 : INFO : Stop word may
2018-08-03 15:12:27,123 : INFO : Stop word may
2018-08-03 15:12:27,124 : INFO : Stop word amount
2018-08-03 15:12:27,125 : INFO : Stop word amount
2018-08-03 15:12:27,127 : INFO : Stop word amount
2018-08-03 15:12:27,129 : INFO : Stop word one
2018-08-03 15:12:27,130 : INFO : Stop word two
2018-08-03 15:12:27,132 : INFO : Stop word made
2018-08-03 15:12:27,133 : INFO : Stop word amount
2018-08-03 15:12:27,134 : INFO : Stop word amount
2018-08-03 15:12:27,135 : INFO : Stop word amount
2018-08-03 15:12:27,136 : INFO : Stop word along
2018-08-03 15:12:27,138 : INFO : Stop word amount
2018-08-03 15:12:27,140 : INFO : Stop word due
2018-08-03 15:12:27,141 : INFO : Stop word may
2018-08-03 15:12:27,142 : INFO : Stop word amount
2018-08-03 15:12:27,143 : INFO : Stop word still
2018-08-03 15:12:27,144 : INFO : Stop word must
2018-08-03 15:12:27,146 

2018-08-03 15:12:29,152 : INFO : Stop word may
2018-08-03 15:12:29,154 : INFO : Stop word either
2018-08-03 15:12:29,156 : INFO : Stop word may
2018-08-03 15:12:29,158 : INFO : Stop word show
2018-08-03 15:12:29,160 : INFO : Stop word may
2018-08-03 15:12:29,161 : INFO : Stop word due
2018-08-03 15:12:29,163 : INFO : Stop word used
2018-08-03 15:12:29,164 : INFO : Stop word unless
2018-08-03 15:12:29,166 : INFO : Stop word becomes
2018-08-03 15:12:29,168 : INFO : Stop word may
2018-08-03 15:12:29,169 : INFO : Stop word amount
2018-08-03 15:12:29,171 : INFO : Stop word also
2018-08-03 15:12:29,173 : INFO : Stop word also
2018-08-03 15:12:29,175 : INFO : Stop word first
2018-08-03 15:12:29,177 : INFO : Stop word full
2018-08-03 15:12:29,178 : INFO : Stop word due
2018-08-03 15:12:29,179 : INFO : Stop word least
2018-08-03 15:12:29,181 : INFO : Stop word full
2018-08-03 15:12:29,182 : INFO : Stop word due
2018-08-03 15:12:29,184 : INFO : Stop word next
2018-08-03 15:12:29,185 : INFO : Sto

2018-08-03 15:12:29,398 : INFO : Stop word us
2018-08-03 15:12:29,400 : INFO : Stop word may
2018-08-03 15:12:29,401 : INFO : Stop word without
2018-08-03 15:12:29,402 : INFO : Stop word may
2018-08-03 15:12:29,404 : INFO : Stop word third
2018-08-03 15:12:29,405 : INFO : Stop word third
2018-08-03 15:12:29,407 : INFO : Stop word may
2018-08-03 15:12:29,410 : INFO : Stop word you
2018-08-03 15:12:29,411 : INFO : Stop word also
2018-08-03 15:12:29,412 : INFO : Stop word us
2018-08-03 15:12:29,413 : INFO : Stop word move
2018-08-03 15:12:29,415 : INFO : Stop word may
2018-08-03 15:12:29,417 : INFO : Stop word call
2018-08-03 15:12:29,418 : INFO : Stop word using
2018-08-03 15:12:29,421 : INFO : Stop word us
2018-08-03 15:12:29,422 : INFO : Stop word either
2018-08-03 15:12:29,423 : INFO : Stop word call
2018-08-03 15:12:29,425 : INFO : Stop word us
2018-08-03 15:12:29,427 : INFO : Stop word may
2018-08-03 15:12:29,428 : INFO : Stop word may
2018-08-03 15:12:29,430 : INFO : Stop word make

2018-08-03 15:12:29,672 : INFO : Stop word amount
2018-08-03 15:12:29,674 : INFO : Stop word must
2018-08-03 15:12:29,675 : INFO : Stop word two
2018-08-03 15:12:29,677 : INFO : Stop word must
2018-08-03 15:12:29,678 : INFO : Stop word also
2018-08-03 15:12:29,680 : INFO : Stop word already
2018-08-03 15:12:29,681 : INFO : Stop word must
2018-08-03 15:12:29,683 : INFO : Stop word either
2018-08-03 15:12:29,684 : INFO : Stop word whether
2018-08-03 15:12:29,686 : INFO : Stop word can
2018-08-03 15:12:29,687 : INFO : Stop word not
2018-08-03 15:12:29,689 : INFO : Stop word amount
2018-08-03 15:12:29,691 : INFO : Stop word amount
2018-08-03 15:12:29,692 : INFO : Stop word may
2018-08-03 15:12:29,693 : INFO : Stop word may
2018-08-03 15:12:29,694 : INFO : Stop word amount
2018-08-03 15:12:29,696 : INFO : Stop word amount
2018-08-03 15:12:29,697 : INFO : Stop word amount
2018-08-03 15:12:29,699 : INFO : Stop word one
2018-08-03 15:12:29,700 : INFO : Stop word two
2018-08-03 15:12:29,701 : I

In [41]:
time_slice =  list(map(lambda doc: len(doc) , docs))
len(docs), time_slice

(3, [1, 1, 1])

In [42]:
docs = docs[0] + docs[1] + docs[2]
#len(docs)
#docs[0]

In [43]:
bigram  =  gensim.models.Phrases(docs)

2018-08-03 15:32:04,557 : INFO : collecting all words and their counts
2018-08-03 15:32:04,559 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-08-03 15:32:04,599 : INFO : collected 8967 word types from a corpus of 12247 words (unigram + bigrams) and 3 sentences
2018-08-03 15:32:04,600 : INFO : using 8967 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [44]:
bigrams = [bigram[line] for line in docs]
dictionary = Dictionary(bigrams)
corpus = [dictionary.doc2bow(doc) for doc in bigrams]

2018-08-03 15:32:10,339 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-03 15:32:10,358 : INFO : built Dictionary(1492 unique tokens: ['a', 'accept', 'acceptance', 'access', 'accompany']...) from 3 documents (total 11055 corpus positions)


In [ ]:
#dictionary.__dict__

In [45]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5)

2018-08-03 15:32:18,100 : INFO : using symmetric eta at 0.2
2018-08-03 15:32:18,101 : INFO : using serial LDA version on this node
2018-08-03 15:32:18,104 : INFO : running online (multi-pass) LDA training, 5 topics, 10 passes over the supplied corpus of 3 documents, updating model once every 3 documents, evaluating perplexity every 3 documents, iterating 50x with a convergence threshold of 0.001000
2018-08-03 15:32:18,164 : INFO : -8.240 per-word bound, 302.3 perplexity estimate based on a held-out corpus of 3 documents with 11055 words
2018-08-03 15:32:18,165 : INFO : PROGRESS: pass 0, at document #3/3
2018-08-03 15:32:18,189 : INFO : topic #0 (0.010): 0.031*"account" + 0.013*"fee" + 0.012*"payment" + 0.012*"agreement" + 0.011*"arbitration" + 0.010*"if" + 0.010*"credit" + 0.009*"claim" + 0.008*"the" + 0.008*"include"
2018-08-03 15:32:18,195 : INFO : topic #1 (0.010): 0.025*"account" + 0.013*"fee" + 0.012*"arbitration" + 0.010*"payment" + 0.009*"credit" + 0.009*"if" + 0.009*"claim" + 0

2018-08-03 15:32:18,669 : INFO : topic #4 (0.010): 0.003*"account" + 0.002*"payment" + 0.002*"fee" + 0.002*"arbitration" + 0.002*"agreement" + 0.002*"credit" + 0.002*"right" + 0.002*"if" + 0.002*"the" + 0.001*"charge"
2018-08-03 15:32:18,674 : INFO : topic diff=0.156075, rho=0.377964
2018-08-03 15:32:18,714 : INFO : -6.633 per-word bound, 99.2 perplexity estimate based on a held-out corpus of 3 documents with 11055 words
2018-08-03 15:32:18,720 : INFO : PROGRESS: pass 6, at document #3/3
2018-08-03 15:32:18,748 : INFO : topic #0 (0.010): 0.033*"account" + 0.019*"fee" + 0.015*"arbitration" + 0.013*"payment" + 0.013*"bank" + 0.012*"card" + 0.011*"if" + 0.010*"credit_one" + 0.010*"agreement" + 0.010*"credit"
2018-08-03 15:32:18,750 : INFO : topic #1 (0.010): 0.005*"account" + 0.003*"fee" + 0.003*"arbitration" + 0.002*"payment" + 0.002*"credit" + 0.002*"if" + 0.002*"claim" + 0.002*"agreement" + 0.002*"the" + 0.002*"charge"
2018-08-03 15:32:18,752 : INFO : topic #2 (0.010): 0.026*"account" 

2018-08-03 15:32:48,645 : INFO : Computing bound, all times
2018-08-03 15:32:48,715 : INFO : initial sslm bound is 116350.624329
2018-08-03 15:32:48,927 : INFO : Computing bound, all times
2018-08-03 15:32:49,013 : INFO : iteration 1 iteration lda seq bound is 116364.128429 convergence is 0.000116
2018-08-03 15:32:49,229 : INFO : Computing bound, all times
2018-08-03 15:32:49,311 : INFO : iteration 2 iteration lda seq bound is 116363.492880 convergence is 0.000005
2018-08-03 15:32:49,345 : INFO : iteration 0 iteration lda seq bound is 449936.598289 convergence is inf
2018-08-03 15:32:49,347 : INFO :  EM iter 1
2018-08-03 15:32:49,349 : INFO : E Step
2018-08-03 15:32:49,350 : INFO : using symmetric eta at 0.2
2018-08-03 15:32:49,353 : INFO : using serial LDA version on this node
2018-08-03 15:32:54,669 : INFO : M Step
2018-08-03 15:32:54,670 : INFO : Fitting topic number 0
2018-08-03 15:32:54,752 : INFO : Computing bound, all times
2018-08-03 15:32:54,835 : INFO : initial sslm bound is 

2018-08-03 15:34:02,258 : INFO : Computing bound, all times
2018-08-03 15:34:02,345 : INFO : iteration 1 iteration lda seq bound is 94350.403170 convergence is 0.000066
2018-08-03 15:34:05,737 : INFO : Computing bound, all times
2018-08-03 15:34:05,822 : INFO : iteration 2 iteration lda seq bound is 94342.525606 convergence is 0.000083
2018-08-03 15:34:05,862 : INFO : Fitting topic number 4
2018-08-03 15:34:05,973 : INFO : Computing bound, all times
2018-08-03 15:34:06,058 : INFO : initial sslm bound is 116388.341162
2018-08-03 15:34:06,223 : INFO : Computing bound, all times
2018-08-03 15:34:06,331 : INFO : iteration 1 iteration lda seq bound is 116388.082243 convergence is 0.000002
2018-08-03 15:34:06,523 : INFO : Computing bound, all times
2018-08-03 15:34:06,615 : INFO : iteration 2 iteration lda seq bound is 116387.866787 convergence is 0.000002
2018-08-03 15:34:06,650 : INFO : iteration 3 iteration lda seq bound is 452652.072885 convergence is 0.000132
2018-08-03 15:34:06,651 : I

2018-08-03 15:35:05,643 : INFO : Fitting topic number 2
2018-08-03 15:35:05,755 : INFO : Computing bound, all times
2018-08-03 15:35:05,823 : INFO : initial sslm bound is 99345.941227
2018-08-03 15:35:08,774 : INFO : Computing bound, all times
2018-08-03 15:35:08,858 : INFO : iteration 1 iteration lda seq bound is 99337.285856 convergence is 0.000087
2018-08-03 15:35:11,949 : INFO : Computing bound, all times
2018-08-03 15:35:12,032 : INFO : iteration 2 iteration lda seq bound is 99329.356197 convergence is 0.000080
2018-08-03 15:35:12,067 : INFO : Fitting topic number 3
2018-08-03 15:35:12,177 : INFO : Computing bound, all times
2018-08-03 15:35:12,243 : INFO : initial sslm bound is 98014.078203
2018-08-03 15:35:15,767 : INFO : Computing bound, all times
2018-08-03 15:35:15,831 : INFO : iteration 1 iteration lda seq bound is 98005.642318 convergence is 0.000086
2018-08-03 15:35:18,987 : INFO : Computing bound, all times
2018-08-03 15:35:19,063 : INFO : iteration 2 iteration lda seq bo

2018-08-03 15:36:20,217 : INFO : Computing bound, all times
2018-08-03 15:36:20,304 : INFO : iteration 2 iteration lda seq bound is 104417.796406 convergence is 0.000048
2018-08-03 15:36:20,337 : INFO : Fitting topic number 1
2018-08-03 15:36:20,432 : INFO : Computing bound, all times
2018-08-03 15:36:20,508 : INFO : initial sslm bound is 107673.262074
2018-08-03 15:36:20,955 : INFO : Computing bound, all times
2018-08-03 15:36:21,016 : INFO : iteration 1 iteration lda seq bound is 107670.498705 convergence is 0.000026
2018-08-03 15:36:21,435 : INFO : Computing bound, all times
2018-08-03 15:36:21,523 : INFO : iteration 2 iteration lda seq bound is 107669.883778 convergence is 0.000006
2018-08-03 15:36:21,556 : INFO : Fitting topic number 2
2018-08-03 15:36:21,684 : INFO : Computing bound, all times
2018-08-03 15:36:21,780 : INFO : initial sslm bound is 102214.589563
2018-08-03 15:36:24,875 : INFO : Computing bound, all times
2018-08-03 15:36:24,968 : INFO : iteration 1 iteration lda s

2018-08-03 15:37:29,780 : INFO : iteration 11 iteration lda seq bound is 453852.681417 convergence is 0.000305
2018-08-03 15:37:29,782 : INFO :  EM iter 12
2018-08-03 15:37:29,786 : INFO : E Step
2018-08-03 15:37:29,791 : INFO : using symmetric eta at 0.2
2018-08-03 15:37:29,796 : INFO : using serial LDA version on this node
2018-08-03 15:37:34,167 : INFO : M Step
2018-08-03 15:37:34,168 : INFO : Fitting topic number 0
2018-08-03 15:37:34,284 : INFO : Computing bound, all times
2018-08-03 15:37:34,367 : INFO : initial sslm bound is 106757.312632
2018-08-03 15:37:37,124 : INFO : Computing bound, all times
2018-08-03 15:37:37,207 : INFO : iteration 1 iteration lda seq bound is 106754.371344 convergence is 0.000028
2018-08-03 15:37:40,394 : INFO : Computing bound, all times
2018-08-03 15:37:40,480 : INFO : iteration 2 iteration lda seq bound is 106751.736771 convergence is 0.000025
2018-08-03 15:37:40,514 : INFO : Fitting topic number 1
2018-08-03 15:37:40,623 : INFO : Computing bound, al

2018-08-03 15:38:46,446 : INFO : Computing bound, all times
2018-08-03 15:38:46,506 : INFO : iteration 2 iteration lda seq bound is 104663.409662 convergence is 0.000014
2018-08-03 15:38:46,530 : INFO : Fitting topic number 4
2018-08-03 15:38:46,599 : INFO : Computing bound, all times
2018-08-03 15:38:46,685 : INFO : initial sslm bound is 103034.431180
2018-08-03 15:38:47,370 : INFO : Computing bound, all times
2018-08-03 15:38:47,452 : INFO : iteration 1 iteration lda seq bound is 103033.909242 convergence is 0.000005
2018-08-03 15:38:48,133 : INFO : Computing bound, all times
2018-08-03 15:38:48,208 : INFO : iteration 2 iteration lda seq bound is 103033.408721 convergence is 0.000005
2018-08-03 15:38:48,230 : INFO : iteration 14 iteration lda seq bound is 454253.844260 convergence is 0.000285
2018-08-03 15:38:48,231 : INFO :  EM iter 15
2018-08-03 15:38:48,233 : INFO : E Step
2018-08-03 15:38:48,238 : INFO : using symmetric eta at 0.2
2018-08-03 15:38:48,241 : INFO : using serial LDA

2018-08-03 15:39:53,139 : INFO : initial sslm bound is 107272.941789
2018-08-03 15:39:55,853 : INFO : Computing bound, all times
2018-08-03 15:39:55,907 : INFO : iteration 1 iteration lda seq bound is 107271.170255 convergence is 0.000017
2018-08-03 15:39:58,889 : INFO : Computing bound, all times
2018-08-03 15:39:58,973 : INFO : iteration 2 iteration lda seq bound is 107269.478760 convergence is 0.000016
2018-08-03 15:39:59,005 : INFO : Fitting topic number 3
2018-08-03 15:39:59,120 : INFO : Computing bound, all times
2018-08-03 15:39:59,206 : INFO : initial sslm bound is 105673.083065
2018-08-03 15:40:02,861 : INFO : Computing bound, all times
2018-08-03 15:40:02,932 : INFO : iteration 1 iteration lda seq bound is 105672.057730 convergence is 0.000010
2018-08-03 15:40:05,798 : INFO : Computing bound, all times
2018-08-03 15:40:05,850 : INFO : iteration 2 iteration lda seq bound is 105671.254832 convergence is 0.000008
2018-08-03 15:40:05,871 : INFO : Fitting topic number 4
2018-08-03

2018-08-03 15:41:08,820 : INFO : Computing bound, all times
2018-08-03 15:41:08,901 : INFO : initial sslm bound is 101431.340824
2018-08-03 15:41:09,681 : INFO : Computing bound, all times
2018-08-03 15:41:09,768 : INFO : iteration 1 iteration lda seq bound is 101430.797522 convergence is 0.000005
2018-08-03 15:41:10,376 : INFO : Computing bound, all times
2018-08-03 15:41:10,458 : INFO : iteration 2 iteration lda seq bound is 101430.322545 convergence is 0.000005
2018-08-03 15:41:10,491 : INFO : Fitting topic number 2
2018-08-03 15:41:10,599 : INFO : Computing bound, all times
2018-08-03 15:41:10,686 : INFO : initial sslm bound is 108099.498638
2018-08-03 15:41:13,100 : INFO : Computing bound, all times
2018-08-03 15:41:13,188 : INFO : iteration 1 iteration lda seq bound is 108098.554034 convergence is 0.000009
2018-08-03 15:41:15,809 : INFO : Computing bound, all times
2018-08-03 15:41:15,890 : INFO : iteration 2 iteration lda seq bound is 108097.654788 convergence is 0.000008
2018-0

In [53]:
ldaseq.print_topics(time=1)

[[('bank', 0.026577728861815676),
  ('credit_one', 0.02175842223574983),
  ('card', 0.018075380079685937),
  ('fee', 0.015388645084681843),
  ('credit_limit', 0.014140982234262455),
  ('account', 0.012658931117405696),
  ('authorized_user', 0.011425060508333392),
  ('arbitration', 0.010451246793550695),
  ('include', 0.009494931286695102),
  ('cardholder', 0.008108238888091287),
  ('advance', 0.007609182398183015),
  ('statement', 0.007416622163030712),
  ('receive', 0.006858075573643085),
  ('annual_membership', 0.006555698808039883),
  ('claims', 0.006526156657350598),
  ('impose', 0.0065196968393091085),
  ('arbitration_administrator', 0.006420866359275078),
  ('relate', 0.006325493601213762),
  ('payment', 0.00629875156486487),
  ('information', 0.006152926993478393)],
 [('account', 0.0466444255378029),
  ('agreement', 0.021968976953928236),
  ('if', 0.02135308666259405),
  ('credit', 0.01870304310188187),
  ('the', 0.016947506958832074),
  ('claim', 0.015464116259912408),
  ('arbi

In [57]:
ldaseq.print_topic_times(topic=4) # evolution of 1st topic

[[('account', 0.05799000349158907),
  ('fee', 0.029769778361043452),
  ('payment', 0.02408516775756787),
  ('arbitration', 0.021992481031803762),
  ('if', 0.018824197571222848),
  ('charge', 0.018103728490225696),
  ('agreement', 0.018053866328498282),
  ('right', 0.016266735885766336),
  ('the', 0.013851123691052738),
  ('credit', 0.013347632111642636),
  ('transaction', 0.012250601282533727),
  ('claim', 0.011349387162321697),
  ('statement', 0.009709868038255232),
  ('balance', 0.00953828817570786),
  ('include', 0.00922185779873414),
  ('pay', 0.0088442964064437),
  ('provide', 0.008512950750449793),
  ('use', 0.00803665213553514),
  ('party', 0.00801910888380635),
  ('information', 0.00793791322048691)],
 [('account', 0.05824545676363376),
  ('fee', 0.029958006987504934),
  ('payment', 0.024162970551232787),
  ('arbitration', 0.02198918321382886),
  ('if', 0.018913097738501604),
  ('charge', 0.01814723542656106),
  ('agreement', 0.018113153788725245),
  ('right', 0.016328348032475

In [52]:
words = [dictionary[word_id] for word_id, count in corpus[1]]
print (words)

['a', 'accept', 'accompany', 'accordance', 'account', 'accrue', 'act', 'action', 'activity', 'add', 'addition', 'additional', 'address', 'advance', 'adverse', 'advertisement', 'affiliate', 'after', 'agency', 'agree', 'agreement', 'all', 'allocate', 'allow', 'american', 'amount', 'an', 'and', 'anew', 'annual', 'annual_percentage', 'any', 'appeal', 'appear', 'applicable', 'applicable_law', 'application', 'apply', 'appoint', 'approval', 'apr', 'arbitrate', 'arbitration', 'arbitration_administrator', 'arbitration_provision', 'arbitrator', 'arbitrator_shall', 'are', 'arise', 'as', 'assess', 'assign', 'assignment', 'association', 'at', 'atm', 'attempt', 'attend', 'attorney', 'authority', 'authorize', 'authorize_user', 'authorized_user', 'auto', 'available', 'avenue', 'average_daily', 'avoid', 'award', 'b', 'balance', 'bank', 'bankruptcy', 'base', 'basis', 'be', 'bear', 'begin', 'behalf', 'believe', 'benefit', 'between', 'bill', 'bill_correct', 'billing', 'billing_cycle', 'billing_period', 'b

In [65]:
doc = ldaseq.doc_topics(2) # check the 558th document in the corpuses topic distribution
print (doc)

[0.00707534 0.19205744 0.00267655 0.49557929 0.30261138]


In [74]:
doc_1 = words = [dictionary[word_id] for word_id, count in corpus[1]]
doc_1 = dictionary.doc2bow(doc_1)
doc_1 = ldaseq[doc_1]
print (doc_1)

doc_2 = words = [dictionary[word_id] for word_id, count in corpus[2]]
doc_2 = dictionary.doc2bow(doc_2)

doc_2 = ldaseq[doc_2]
print (doc_2)

2018-08-03 15:51:36,519 : INFO : using symmetric eta at 0.2
2018-08-03 15:51:36,521 : INFO : using serial LDA version on this node
2018-08-03 15:51:41,084 : INFO : using symmetric eta at 0.2
2018-08-03 15:51:41,086 : INFO : using serial LDA version on this node


[0.01057609 0.5387053  0.14997828 0.00756016 0.29318017]
[1.63548897e-02 3.99640993e-01 1.04816309e-05 1.53765779e-01
 4.30227857e-01]


In [75]:
from gensim.matutils import hellinger
hellinger(doc_1, doc_2)

0.36380348310901595

In [79]:
doc_governemt_1 = ['tony', 'stark', 'house', 'burn', 'ground' , 'ultron']
doc_governemt_1 = dictionary.doc2bow(doc_governemt_1)
doc_governemt_1 = ldaseq[doc_governemt_1]

hellinger(doc_2, doc_governemt_1)

2018-08-03 15:53:29,179 : INFO : using symmetric eta at 0.2
2018-08-03 15:53:29,185 : INFO : using serial LDA version on this node


0.4939022704227056